# Graph isomorphism

Ising formulation here: https://arxiv.org/abs/1302.5843 (Lucas 2014)

Lets assume that we have two directed graphs $G_1 = (V_1, E_1)$ and $G_2 = (V_2, E_2)$. They are isomorphic if we can find bijection from $V_1$ to $V_2$ so that structure of mapped $V_1$ is identical to $V_2$. Obviously $n=|V_1|=|V_2|$. 

Approach is to form QUBO which elements are mappings of single vertex $(v_1 \in V_1, v_2 \in V_2)$. So there are altogether $n^2$ elements. Following constraints are build to QUBO: 
1. every vertex $v \in G_1$ must be mapped to vertex $v \in G_2$ 
2. every vertex $v \in G_2$ must be mapped from vertex $v \in G_1$
3. with mapping every $e \in E_1$ should be mapped to $e \in E_2$ 
4. with mapping every $e \in E_2$ should be mapped to $e \in E_1$(this is however not needed, 1 and 2 ensures bijection already) 

Lowest energylevel should be $-|E|$. If this is reached, graphs are isomorphic.

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector
import networkx as nx
from networkx.classes.function import path_weight
import random

## QUBO function

In [2]:
def create_qubo(E1,E2,vertices,p):
    Q = np.zeros((vertices*vertices, vertices*vertices))
    
    # Constraint 1: penalty if several mappings from same source
    for i in range(vertices): 
        for j in range(vertices): 
            for k in range(j+1,vertices): 
                Q[i*vertices+j,i*vertices+k]=p 

    # Constaint 2: penalty if several mappings to same target
    for i in range(vertices): 
        for j in range(vertices): 
            for k in range(j+1,vertices): 
                Q[i+vertices*j,i+vertices*k]=p 
                
    # Constraint 3: -1 for each succesfully mapped edge
    for e1 in E1: 
        for e2 in E2: 
            Q[e1[0]*vertices+e2[0], e1[1]*vertices+e2[1]] -= 1 
            
    return Q

## Helper functions

In [82]:
def result_info(sample, e):
    print('Lowest energy should be:',-e)
    print('Lowest energy was:',int(sample.energy))
    if -e==int(sample.energy):
        print('Graphs are isomorphic')
    else:
        print('Graphs are NOT isomorphic')
    m = ''
    res = {}
    for k,v in sample.sample.items():
        if v==1:
            m += str(k)+', '
            res[k[0]]=k[1]
    print('Mapping: '+m)
    return res
    
def solve_gi(solver, num_reads, E1, E2, vertices, p, qpu_info=False):
    Q = create_qubo(E1,E2,vertices,p)

    labels = {}
    for i in range(vertices):
        for j in range(vertices):
            labels[i*vertices+j] = (i,j)
        
    bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
    bqm = bqm.relabel_variables(labels, inplace=False)
    sampleset = solver.sample(bqm, num_reads=num_reads).aggregate()
    res = result_info(sampleset.first, len(E1))
    if qpu_info:
        print('\nNumber of logical qubits:',Q.shape[0])
        print('Number of couplers:', len(bqm.quadratic))
        qpu_time = sampleset.info['timing']['qpu_access_time'] / 1000
        qubits = sum(len(x) for x in sampleset.info['embedding_context']['embedding'].values())
        print('QPU time used (ms): {:.1f}'.format(qpu_time))
        print('Physical qubits used: {}\n'.format(qubits))
    return res

## Simple graphs

In [77]:
vertices = 5
E1 = np.array([(0, 2), (2, 1), (1, 3), (3, 2), (0, 1), (3, 4), (2, 4)])
E3 = np.array([(0, 2), (2, 1), (1, 3), (3, 2), (0, 1), (3, 4), (2, 0)]) # last number difference

perm={0:2, 1:1, 2:3, 3:4, 4:0}
E2 = np.array(E1, copy=True) 
for i in range(len(E2)):
    E2[i]=(perm[E2[i][0]],perm[E2[i][1]])
    
p = len(E2)
print('Penalty:',p)

Penalty: 7


### Heuristic solver

In [5]:
print('Identical graphs')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E1, E1, vertices, p)

Identical graphs
Lowest energy should be: -7
Lowest energy was: -7
Graphs are isomorphic
Mapping: (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), 


In [6]:
print('Differing graphs')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E1, E3, vertices, p)

Differing graphs
Lowest energy should be: -7
Lowest energy was: -6
Graphs are NOT isomorphic
Mapping: (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), 


In [7]:
print('Identical but nodes permuted')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E1, E2, vertices, p)

Identical but nodes permuted
Lowest energy should be: -7
Lowest energy was: -7
Graphs are isomorphic
Mapping: (0, 2), (1, 1), (2, 3), (3, 4), (4, 0), 


### Quantum solver

In [8]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [83]:
print('Identical graphs')
res = solve_gi(EmbeddingComposite(machine), 200, E1, E1, vertices, p, qpu_info=True)

Identical graphs
Lowest energy should be: -7
Lowest energy was: -7
Graphs are isomorphic
Mapping: (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), 

Number of logical qubits: 25
Number of couplers: 149
QPU time used (ms): 45.8
Physical qubits used: 66



In [84]:
print('Differing graphs')
res = solve_gi(EmbeddingComposite(machine), 200, E1, E3, vertices, p, qpu_info=True)

Differing graphs
Lowest energy should be: -7
Lowest energy was: -6
Graphs are NOT isomorphic
Mapping: (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), 

Number of logical qubits: 25
Number of couplers: 149
QPU time used (ms): 38.4
Physical qubits used: 68



In [85]:
print('Identical but nodes permuted')
res = solve_gi(EmbeddingComposite(machine), 200, E1, E2, vertices, p, qpu_info=True)

Identical but nodes permuted
Lowest energy should be: -7
Lowest energy was: -7
Graphs are isomorphic
Mapping: (0, 2), (1, 1), (2, 3), (3, 4), (4, 0), 

Number of logical qubits: 25
Number of couplers: 149
QPU time used (ms): 51.2
Physical qubits used: 72



## Larger graph

In [86]:
seed = 42
vertices2 = 10
random.seed(seed)
G = nx.gnp_random_graph(vertices2, 0.30, seed, directed=True)
E4 = [] 
for e in G.edges(data=True):
    E4.append((e[0],e[1]))
print('Number of edges:',len(E4))
print('Number of vertices:',vertices2)
p = len(E4)
print('Penalty:',p)

Number of edges: 34
Number of vertices: 10
Penalty: 34


In [87]:
mapping = dict(zip(G.nodes(), sorted(G.nodes(), key=lambda k: random.random())))
print(mapping)
G2 = nx.relabel_nodes(G, mapping)
E5 = []
for e in G2.edges(data=True):
    E5.append((e[0],e[1]))

{0: 1, 1: 9, 2: 7, 3: 3, 4: 2, 5: 8, 6: 0, 7: 5, 8: 4, 9: 6}


In [88]:
G3 = nx.gnp_random_graph(vertices2, 0.30, seed+1, directed=True)
E6 = [] 
for e in G3.edges(data=True):
    E6.append((e[0],e[1]))

In [89]:
print('Identical graphs')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E4, E4, vertices2, p)

Identical graphs
Lowest energy should be: -34
Lowest energy was: -34
Graphs are isomorphic
Mapping: (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), 


In [90]:
print('Similar graphs')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E4, E5, vertices2, p)

Similar graphs
Lowest energy should be: -34
Lowest energy was: -34
Graphs are isomorphic
Mapping: (0, 1), (1, 9), (2, 7), (3, 3), (4, 2), (5, 8), (6, 0), (7, 5), (8, 4), (9, 6), 


In [91]:
print('Different graphs')
res = solve_gi(SimulatedAnnealingSampler(), 1000, E4, E6, vertices2, p)

Different graphs
Lowest energy should be: -34
Lowest energy was: -18
Graphs are NOT isomorphic
Mapping: (0, 8), (1, 7), (2, 5), (3, 9), (4, 2), (5, 3), (6, 4), (7, 0), (8, 1), (9, 6), 


## Testing procedure

In [92]:
for seed in range(20):
    vertices = 10
    random.seed(seed)
    G1 = nx.gnp_random_graph(vertices2, 0.30, seed, directed=True)
    E1 = [] 
    for e in G1.edges(data=True):
        E1.append((e[0],e[1]))
    p = len(E1)
    
    mapping = dict(zip(G.nodes(), sorted(G.nodes(), key=lambda k: random.random())))
    G2 = nx.relabel_nodes(G1, mapping)
    E2 = []
    for e in G2.edges(data=True):
        E2.append((e[0],e[1]))
        
    res = solve_gi(SimulatedAnnealingSampler(), 1000, E1, E2, vertices, p)
    if res==mapping:
        print('Result ok\n')
    else:
        print('ERROR\n')

Lowest energy should be: -17
Lowest energy was: -17
Graphs are isomorphic
Mapping: (0, 3), (1, 7), (2, 5), (3, 2), (4, 8), (5, 4), (6, 9), (7, 1), (8, 6), (9, 0), 
Result ok

Lowest energy should be: -26
Lowest energy was: -26
Graphs are isomorphic
Mapping: (0, 9), (1, 8), (2, 0), (3, 3), (4, 5), (5, 4), (6, 6), (7, 2), (8, 7), (9, 1), 
Result ok

Lowest energy should be: -19
Lowest energy was: -19
Graphs are isomorphic
Mapping: (0, 2), (1, 3), (2, 7), (3, 8), (4, 9), (5, 6), (6, 5), (7, 4), (8, 1), (9, 0), 
Result ok

Lowest energy should be: -23
Lowest energy was: -23
Graphs are isomorphic
Mapping: (0, 6), (1, 5), (2, 9), (3, 0), (4, 8), (5, 2), (6, 1), (7, 3), (8, 4), (9, 7), 
Result ok

Lowest energy should be: -30
Lowest energy was: -30
Graphs are isomorphic
Mapping: (0, 4), (1, 1), (2, 3), (3, 9), (4, 0), (5, 2), (6, 5), (7, 8), (8, 7), (9, 6), 
Result ok

Lowest energy should be: -29
Lowest energy was: -29
Graphs are isomorphic
Mapping: (0, 6), (1, 7), (2, 0), (3, 9), (4, 4), (5